In [1]:
%run init.ipynb

Using TensorFlow backend.
/home/fanyixing/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/fanyixing/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


matchzoo version 2.1.0

data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
loading embedding ...
embedding loaded as `glove_embedding`


In [2]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, fixed_length_right=100, remove_stop_words=False)
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 3718.40it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:07<00:00, 2381.54it/s]
Building Vocabulary from a datapack.: 100%|██████████| 404415/404415 [00:00<00:00, 1847527.98it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 3771.43it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:07<00:00, 2367.53it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 32836.35it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 3487.84it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00

In [3]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7faf25c75f98>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7faf264499b0>,
 'vocab_size': 16674,
 'embedding_input_dim': 16674,
 'input_shapes': [(10,), (100,)]}

In [4]:
ranking_task = mz.tasks.Ranking(loss=mz.losses.RankCrossEntropyLoss(num_neg=10))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [5]:
bin_size = 30
model = mz.models.DRMM()
model.params.update(preprocessor.context)
model.params['input_shapes'] = [[10,], [10, bin_size,]]
model.params['task'] = ranking_task
model.params['mask_value'] = 0
model.params['embedding_output_dim'] = glove_embedding.output_dim
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 10
model.params['mlp_num_fan_out'] = 1
model.params['mlp_activation_func'] = 'tanh'
model.params['optimizer'] = 'adadelta'
model.build()
model.compile()
model.backend.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 10)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 10, 300)      5002200     text_left[0][0]                  
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 10, 1)        300         embedding[0][0]                  
__________________________________________________________________________________________________
match_histogram (InputLayer)    (None, 10, 30)       0                                            
__________________________________________________________________________________________________
attention_

In [6]:
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])
# normalize the word embedding for fast histogram generating.
l2_norm = np.sqrt((embedding_matrix*embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]
model.load_embedding_matrix(embedding_matrix)

In [7]:
hist_callback = mz.data_generator.callbacks.Histogram(embedding_matrix, bin_size=30, hist_mode='LCH')

In [8]:
pred_generator = mz.DataGenerator(test_pack_processed, mode='point', callbacks=[hist_callback])
pred_x, pred_y = pred_generator[:]
evaluate = mz.callbacks.EvaluateAllMetrics(model, 
                                           x=pred_x, 
                                           y=pred_y, 
                                           once_every=1, 
                                           batch_size=len(pred_y),
                                           model_save_path='./drmm_pretrained_model/'
                                          )

In [9]:
train_generator = mz.DataGenerator(train_pack_processed, mode='pair', num_dup=5, num_neg=10, batch_size=20, 
                                   callbacks=[hist_callback])
print('num batches:', len(train_generator))



num batches: 255


In [10]:
history = model.fit_generator(train_generator, epochs=30, callbacks=[evaluate], workers=30, use_multiprocessing=True)

Epoch 1/30
255/255 [==============================] - 29s 113ms/step - loss: 2.2520
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.565716344425806 - normalized_discounted_cumulative_gain@5(0.0): 0.6337418608669659 - mean_average_precision(0.0): 0.5867500331707677
Epoch 2/30
255/255 [==============================] - 44s 172ms/step - loss: 1.9129
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5985553932954132 - normalized_discounted_cumulative_gain@5(0.0): 0.6538053305162407 - mean_average_precision(0.0): 0.6119736749640002
Epoch 3/30
255/255 [==============================] - 44s 172ms/step - loss: 1.5735
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6009435695818752 - normalized_discounted_cumulative_gain@5(0.0): 0.6639328074555028 - mean_average_precision(0.0): 0.6141421880590333
Epoch 4/30
255/255 [==============================] - 44s 174ms/step - loss: 1.3388
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.60611654032328 - normal

In [11]:
drmm_model = mz.load_model('./drmm_pretrained_model/16')
test_generator = mz.DataGenerator(data_pack=dev_pack_processed[:10], mode='point', callbacks=[hist_callback])
test_x, test_y = test_generator[:]
prediction = drmm_model.predict(test_x)
prediction

array([[ 2.1248043 ],
       [-0.7405751 ],
       [ 1.2434225 ],
       [-3.161619  ],
       [-1.5212955 ],
       [-1.3774216 ],
       [-0.43242887],
       [-3.8191142 ],
       [-2.490344  ],
       [ 1.9016179 ]], dtype=float32)

In [12]:
import shutil
shutil.rmtree('./drmm_pretrained_model/')